## RainFall Data Analysis

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

### Download Partitioned Data

In [2]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "../data/rainfall/partitions/"


# Retrieve the article metadata
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want


# Zip to Folder Unzip
files_to_dl = ["data.zip"]  # feel free to add other files here
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])
        
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)
    

### Combine Data

In [3]:
# Read the data
files_to_combine = glob.glob(output_directory + "*.csv")
files_to_combine.remove(output_directory + "observed_daily_rainfall_SYD.csv")
df = pd.concat(
    (pd.read_csv(file, index_col=0)
                .assign(model=re.findall(r'[^\/&\\]+(?=_daily_rainfall_NSW\.)', file)[0])
                for file in files_to_combine)
    )

# Save the combined data
data_path = "../data/rainfall/"
os.makedirs(data_path + "combined/", exist_ok=True)
df.to_csv(data_path + "combined/rainfall_data.csv")

### Perform a simple EDA in R

In [4]:
import pyarrow.dataset as ds
import pyarrow as pa
import pandas as pd
import pyarrow 
from pyarrow import csv
import rpy2_arrow.pyarrow_rarrow as pyra

In [5]:
%load_ext rpy2.ipython

In [6]:
%%R
suppressMessages(library(dplyr, warn.conflicts = FALSE))
suppressMessages(library(arrow, warn.conflicts = FALSE))

In [7]:
%%time
df = pd.read_csv("../data/rainfall/combined/rainfall_data.csv")

CPU times: user 40.6 s, sys: 8.17 s, total: 48.8 s
Wall time: 52.6 s


#### Feather

In [8]:
%%time
df.to_feather(data_path +"combined/rainfall_data.feather")

CPU times: user 6.8 s, sys: 3.09 s, total: 9.89 s
Wall time: 7.13 s


In [9]:
%%time
%%R
ds <- open_dataset("../data/rainfall/combined/rainfall_data.feather", format="feather")

CPU times: user 7.69 ms, sys: 11.1 ms, total: 18.8 ms
Wall time: 139 ms


In [10]:
%%time
%%R
print(ds %>% collect)

# A tibble: 62,467,843 × 7
   time                lat_min lat_max lon_min lon_max `rain (mm/day)` model    
   <chr>                 <dbl>   <dbl>   <dbl>   <dbl>           <dbl> <chr>    
 1 1957-08-09 12:00:00   -33.6   -31.7    143.    145.        6.33e+ 0 MPI-ESM-…
 2 1957-08-10 12:00:00   -33.6   -31.7    143.    145.        9.83e- 1 MPI-ESM-…
 3 1957-08-11 12:00:00   -33.6   -31.7    143.    145.        4.18e+ 0 MPI-ESM-…
 4 1957-08-12 12:00:00   -33.6   -31.7    143.    145.        1.83e- 1 MPI-ESM-…
 5 1957-08-13 12:00:00   -33.6   -31.7    143.    145.        3.71e-13 MPI-ESM-…
 6 1957-08-14 12:00:00   -33.6   -31.7    143.    145.        1.75e- 3 MPI-ESM-…
 7 1957-08-15 12:00:00   -33.6   -31.7    143.    145.        2.89e+ 0 MPI-ESM-…
 8 1957-08-16 12:00:00   -33.6   -31.7    143.    145.        4.10e-13 MPI-ESM-…
 9 1957-08-17 12:00:00   -33.6   -31.7    143.    145.        4.03e-13 MPI-ESM-…
10 1957-08-18 12:00:00   -33.6   -31.7    143.    145.        3.73e-13 MPI-ESM-…
#

#### Arrow exchange

In [11]:
%%time
dataset = ds.dataset("../data/rainfall/combined/rainfall_data.csv", format="csv")
table = dataset.to_table()
r_table = pyra.converter.py2rpy(table)

CPU times: user 34 s, sys: 2.11 s, total: 36.1 s
Wall time: 35.1 s


In [12]:
%%time
%%R -i r_table
start_time <- Sys.time()
suppressMessages(library(dplyr))
result <- r_table 
end_time <- Sys.time()
print(result %>% collect())
print(end_time - start_time)

# A tibble: 62,467,843 × 7
   time                lat_min lat_max lon_min lon_max `rain (mm/day)` model    
   <dttm>                <dbl>   <dbl>   <dbl>   <dbl>           <dbl> <chr>    
 1 1889-01-01 04:00:00   -35.4   -33.6    142.    143.        4.24e-13 MPI-ESM-…
 2 1889-01-02 04:00:00   -35.4   -33.6    142.    143.        4.22e-13 MPI-ESM-…
 3 1889-01-03 04:00:00   -35.4   -33.6    142.    143.        4.50e-13 MPI-ESM-…
 4 1889-01-04 04:00:00   -35.4   -33.6    142.    143.        4.25e-13 MPI-ESM-…
 5 1889-01-05 04:00:00   -35.4   -33.6    142.    143.        4.27e-13 MPI-ESM-…
 6 1889-01-06 04:00:00   -35.4   -33.6    142.    143.        4.20e-13 MPI-ESM-…
 7 1889-01-07 04:00:00   -35.4   -33.6    142.    143.        4.19e-13 MPI-ESM-…
 8 1889-01-08 04:00:00   -35.4   -33.6    142.    143.        4.56e-13 MPI-ESM-…
 9 1889-01-09 04:00:00   -35.4   -33.6    142.    143.        2.53e+ 0 MPI-ESM-…
10 1889-01-10 04:00:00   -35.4   -33.6    142.    143.        4.12e- 2 MPI-ESM-…
#

#### Parquet

In [13]:
%%time
df.to_parquet("../data/rainfall/combined/rainfall_data.parquet")

CPU times: user 16.3 s, sys: 4.76 s, total: 21.1 s
Wall time: 20.9 s


In [14]:
%%time
%%R
ds <- open_dataset("../data/rainfall/combined/rainfall_data.parquet", format="parquet")

CPU times: user 6.93 ms, sys: 9.21 ms, total: 16.1 ms
Wall time: 17.1 ms


In [15]:
%%time
%%R
print(ds %>% collect)

# A tibble: 62,467,843 × 7
   time                lat_min lat_max lon_min lon_max `rain (mm/day)` model    
   <chr>                 <dbl>   <dbl>   <dbl>   <dbl>           <dbl> <chr>    
 1 1987-12-10 12:00:00   -35.4   -33.6    143.    145.        17.5     AWI-ESM-…
 2 1987-12-11 12:00:00   -35.4   -33.6    143.    145.         0.0153  AWI-ESM-…
 3 1987-12-12 12:00:00   -35.4   -33.6    143.    145.        16.7     AWI-ESM-…
 4 1987-12-13 12:00:00   -35.4   -33.6    143.    145.         1.63    AWI-ESM-…
 5 1987-12-14 12:00:00   -35.4   -33.6    143.    145.         0.897   AWI-ESM-…
 6 1987-12-15 12:00:00   -35.4   -33.6    143.    145.         0.109   AWI-ESM-…
 7 1987-12-16 12:00:00   -35.4   -33.6    143.    145.         0.0160  AWI-ESM-…
 8 1987-12-17 12:00:00   -35.4   -33.6    143.    145.         0.00143 AWI-ESM-…
 9 1987-12-18 12:00:00   -35.4   -33.6    143.    145.         0.0213  AWI-ESM-…
10 1987-12-19 12:00:00   -35.4   -33.6    143.    145.         4.07    AWI-ESM-…
#

### We also tried other approaches, the reason that we chose this approach over others is that:
- 1. Arrow exchange: It is slower than feather file. It used around 40 seconds on my laptop.
- 2. Pandas exchange: This approach is the most slow among all methods, I waited about 40 minutes and I gave up because it hadn't stopped running by that time.
- 3. Parquet file: Parquet files are mainly suitable for long term storage whereas feather files are used in short term storage such as dataframe transfers.
Besides, in terms of time, transfer pandas dataframe to parquet took much longer time than to feather file. In terms of storage memory, Parquet files is much smaller than Feather files which can save some memory space for long term storage.